### Requirements

In [ ]:
! pip install selenium

### Bibliotecas

In [ ]:
from selenium import webdriver


### Web-Driver
Lembre-se de colocar o path do executavel do driver

In [ ]:
driver = webdriver.Chrome(executable_path="")

### Setup do Teste


In [ ]:
Oraculo_dicionario = {}

## Casos de Teste

### Caso 1 - Login
1) abrir browser
2) abrir url
3) preencher Username e Senha
4) apertar login
5) inspecionar se o título da página está correto
6) fechar browser


### Caso 2 - teste de credenciais invalidas